# Quadratic Programs

## Introduction

In this tutorial, we briefly introduce how to build optimization problems using Qiskit's optimization module.
Qiskit introduces the `QuadraticProgram` class to make a model of an optimization problem.
More precisely, it deals with quadratically constrained quadratic programs given as follows:

$$
\begin{align}
\text{minimize}\quad& x^\top Q_0 x + c^\top x\\
\text{subject to}\quad& A x \leq b\\
& x^\top Q_i x + a_i^\top x \leq r_i, \quad 1,\dots,i,\dots,q\\
& l_i \leq x_i \leq u_i, \quad 1,\dots,i,\dots,n,
\end{align}
$$

where the $Q_i$ are $n \times n$ matrices, $A$ is a $m \times n$ matrix , $x$, and $c$ are $n$-dimensional vectors, $b$ is an $m$-dimensional vector, and where $x$ can defined as binary, integer, or continuous variables.
In addition to "$\leq$" constraints 'QuadraticProgram' also supports "$\geq$" and "$=$".

## Loading a `Quadratic Program` from an LP file

As setup, you need to import the following module.

In [1]:
from qiskit.optimization import QuadraticProgram

You start with an empty model. How to add variables and constraints to a model is explained in the section "Directly constructing a `QuadraticProgram`".

Qiskit's optimization module supports the conversion from Docplex model. You can easily make a model of an optimization problem with Docplex.
You can find the documentation of Docplex at https://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html

You can load a Docplex model to `QuadraticProgram` by invoking `from_docplex`.

## Loading a `QuadraticProgram` from a docplex model

First, we need to make the DOcplex model

In [2]:
# Let's make a Docplex model!

# Necessary docplex import


# Declare a new model, called "docplex model"

# Add a binary variable called x

# Add an integer variable, y, with lower bound -1 and upper bound 5

# Add an objective function, which we wish to minimize: x + 2 * y

# Add a constraint x - y =3 

# Add the constraint (x+y)*(x-y) <= 1

In [4]:
# Let's print the model, by exporting it to an LP string


Now that we have a model, we can load it into Qiskit's `QuadraticProgram` object.

In [5]:
# Instantiate the QuantumProgram object

# Now, pull in the DOcplex model


In [6]:
# Let's print the model, by exporting it to an LP string


## Directly constructing a `QuadraticProgram`

Above, we made a model by first creating a DOcplex model, and then pulling it into the `QuadraticProgram`.

Now, let's make a model using the `QuadraticProgram` directly.

We start from an empty model.

In [7]:
# Make an empty problem


In [8]:
# Let's print the model, by exporting it to an LP string


The `QuadraticProgram` supports three types of variables:
- Binary variable
- Integer variable
- Continuous variable

When you add variables, you can specify names, types, lower bounds and upper bounds.

When you display your problem as LP format,
`Binaries` denotes binary variables and `Generals` denotes integer variables.
If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones with default lower bound = 0 and upper bound = infinity.
Note that you cannot use 'e' or 'E' as the first character of names due to the [specification of LP format](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_VariableNames.html).

In [9]:
# Let's add some variables!

# First, a binary variable called x

# Second, an integer variable called y, with lowerbound o f -1 and upper bound of 5

# Third, a continuous variable called z, also with lowerbound o f -1 and upper bound of 5


In [10]:
# Let's print the model, by exporting it to an LP string

You can set the objective function by invoking `QuadraticProgram.minimize` or `QuadraticProgram.maximize`.
You can add a constant term as well as linear and quadratic objective function by specifying linear and quadratic terms with either list, matrix or dictionary.

Note that in the LP format the quadratic part has to be scaled by a factor $1/2$.
Thus, when printing as LP format, the quadratic part is first multiplied by 2 and then divided by 2 again.

In [11]:
# Add objective function using dictionaries

# For this objective function, we want to minimize  (x + (2*xy - z^2) + 3)


In [12]:
# Let's print the model, by exporting it to an LP string

In [13]:
# Add objective function using lists/arrays

# Here, we need to encode the array and matrix for the linear and quadratic terms, respectively


In [14]:
# Let's print the model, by exporting it to an LP string

You can access the constant, the linear term, and the quadratic term by looking at `Quadratic.objective.{constant, linear, quadratic}`, respectively.
As for linear and quadratic terms, you can get a dense matrix (`to_array`), a sparse matrix (`coefficients`), and a dictionary (`to_dict`).
For dictionaries, you can specify whether to use variable indices or names as keys.

## Adding/removing linear and quadratic constraints

You can add linear constraints by setting name, linear expression, sense and right-hand-side value (rhs).
You can use senses 'EQ', 'LE', and 'GE' as Docplex supports.

In [15]:
# Let's add 3 linear constraints!
#   x + 2y  = 3 -- call it "lin_eq"

#   x + 2y <= 3 -- call it "lin_leq"


#   x + 2y >= 3 -- call it "lin_geq"

In [16]:
# Let's print the model, by exporting it to an LP string

You can add quadratic constraints as well as objective function and linear constraints.

In [17]:
# Now, add 3 quadratic constraints!

# x^2 - yz + x + y = 1

# x^2 - yz + x + y <= 1


# # x^2 - yz + x + y >= 1 

In [18]:
# Let's print the model, by exporting it to an LP string

You can access linear and quadratic terms of linear and quadratic constraints as in the same way as the objective function.

In [19]:
# Let's get the linear constraint that's a greater-than-or-equal-to ("lin_geq")

In [20]:
# Print the constraint

You can also remove linear/quadratic constraints by `remove_linear_constraint` and `remove_quadratic_constraint`.

In [21]:
# Let's remove the quadratic less-than-or-equal-to constraint ("quad_leq")

In [22]:
# Now, let's print the model

## Substituting Variables

You can substitute some of variables with constants or other variables.
More precisely, `QuadraticProgram` has a method `substitute_variables(constants=..., variables=...)` to deal with the following two cases.
- $x \leftarrow c$: when `constants` have a dictionary `{x: c}`. 
- $x \leftarrow c y$: when `variables` have a dictionary `{x: (y, c)}`.

In [23]:
# Let's make two substitutions:
#  Set the variable x to 0
#  Set the variable y to -z


In [24]:
# Print the model

If the resulting problem is infeasible due to lower bounds or upper bounds, the methods returns the status `Status.INFEASIBLE`.
We try to replace variable `x` with -1, but -1 is out of range of `x` (0 <= `x` <= 1). So, it returns `Status.INFEASIBLE`.

In [25]:
# Let's make a substitution which violates a constraint,
# by setting x = -1.


You cannot substitute variables multiple times. 
The method raises an error in such a case.

In [26]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright